In [1]:
import torch
from torch.autograd import Variable
from torch.nn import CrossEntropyLoss
from torch.nn.functional import one_hot
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from tqdm.notebook import tqdm

from src.federated_learning.torchutils import get_device
from src.models.mnist import CNN, MLP

In [2]:
device = get_device()
print("Using device: " + device)

Using device: cpu


In [3]:
batch_size = 64
lr = 0.01
epochs = 50

In [4]:
train_data = datasets.MNIST(
    root='../../data',
    train=True,
    transform=ToTensor(),
    target_transform=lambda y: one_hot(torch.tensor(y), 10).type(torch.FloatTensor),
    download=True,
)
test_data = datasets.MNIST(
    root='../../data',
    train=False,
    transform=ToTensor(),
    target_transform=lambda y: one_hot(torch.tensor(y), 10).type(torch.FloatTensor),
    download=True
)

In [5]:
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

In [6]:
model = CNN()

In [7]:
loss_fn = CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr = lr)

In [8]:
for t in range(epochs):
    print(f"Epoch {t+1} --------------------------------")
    model.train()
    for batch, (X, y) in enumerate(train_loader):
        X, y = X.to(device), y.to(device)
        b_X = X
        b_y = y
        
        pred = model(b_X)
        loss = loss_fn(pred, b_y)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if batch % 200 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}")
            
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in test_loader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y.argmax(1)).type(torch.float).sum().item()
        test_loss /= len(test_loader)
        correct /= len(test_loader.dataset)
        print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

Epoch 1 --------------------------------
loss: 2.347141
loss: 0.108550
loss: 0.040600
loss: 0.134699
loss: 0.017822
Test Error: 
 Accuracy: 98.4%, Avg loss: 0.055145 

Epoch 2 --------------------------------
loss: 0.060396
loss: 0.033013
loss: 0.191553
loss: 0.173049
loss: 0.139961
Test Error: 
 Accuracy: 98.4%, Avg loss: 0.054648 

Epoch 3 --------------------------------
loss: 0.023944
loss: 0.007080
loss: 0.234555
loss: 0.068044
loss: 0.012941
Test Error: 
 Accuracy: 98.4%, Avg loss: 0.061092 

Epoch 4 --------------------------------
loss: 0.037082
loss: 0.003994
loss: 0.174354
loss: 0.016295
loss: 0.126619
Test Error: 
 Accuracy: 98.7%, Avg loss: 0.047146 

Epoch 5 --------------------------------
loss: 0.005158
loss: 0.002767
loss: 0.015967
loss: 0.076378
loss: 0.057800
Test Error: 
 Accuracy: 98.9%, Avg loss: 0.038626 

Epoch 6 --------------------------------
loss: 0.000414
loss: 0.009920
loss: 0.015950
loss: 0.015575
loss: 0.005259
Test Error: 
 Accuracy: 98.8%, Avg loss: 0.0

KeyboardInterrupt: 